In [22]:
import folium
import geopandas
import numpy as np

In [35]:
gdf = geopandas.read_file('./output/')
data = np.array([[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry ])

In [36]:
m = folium.Map(location=data.mean(axis=0), zoom_start=12)
folium.plugins.HeatMap(data).add_to(m)
m